<a href="https://colab.research.google.com/github/mainguyen1901/SOA_Universities_Web_Scraping/blob/main/Mai_Nguyen_Customer_Sales_Project_(Updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Sales Project

This project involves working with a dataset of customer sales data. There are two versions, one yearly and the other quarterly. See the GroupBy and Preparing Customer Sales Data for the background on their creation.

You are welcome to add cells as needed.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
import altair as alt

In [7]:
# Format the numbers to not display in scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# Read in yearly customer data 
yearly = pd.read_csv('/content/drive/MyDrive/Hollins Data Science /HDS/Datasets/Customer Sales/by_year.csv', parse_dates=[1])
yearly.head()

,CUSTOMER_ID,YEAR,REVENUE,PROFIT,CUSTOMER_TYPE,IS_TREATMENT_FLAG,STATUS,REVENUE_DIFF,CHURN
0,0,2015-01-01,416.41,142.59,Education,NaN,CONT,0.00,True
1,1,2015-01-01,526.05,172.70,Education,NaN,CONT,0.00,False
2,1,2016-01-01,656.59,287.05,Education,NaN,UPSELL,130.54,False
3,1,2017-01-01,912.12,336.87,Education,NaN,UPSELL,255.53,False
4,1,2018-01-01,966.24,519.66,Education,NaN,UPSELL,54.12,False


In [8]:
# Read in quarterly customer data 
quarterly = pd.read_csv('/content/drive/MyDrive/Hollins Data Science /HDS/Datasets/Customer Sales/by_quarter.csv', parse_dates=[1])
quarterly.head()

,CUSTOMER_ID,TRANSACTION_PERIOD,REVENUE,CUSTOMER_TYPE,PROFIT,IS_TREATMENT_FLAG,STATUS,REVENUE_DIFF,CHURN
0,0,2015-03-31,139.46,Education,46.15,NaN,CONT,0.00,False
1,0,2015-06-30,136.07,Education,56.09,NaN,DOWNSELL,-3.39,False
2,0,2015-09-30,140.88,Education,40.35,NaN,UPSELL,4.81,True
3,1,2015-03-31,90.13,Education,31.10,NaN,CONT,0.00,False
4,1,2015-06-30,118.55,Education,53.91,NaN,UPSELL,28.42,False


## Part One

Create a table of yearly revenue, profit and profit margin. Profit margin is profit divided by revenue. Exclude year 2021. 

In [9]:
yearly_df = yearly.loc[yearly['YEAR'] != '2021-01-01', ['YEAR', 'REVENUE', 'PROFIT']]
yearly_df = yearly_df.groupby(['YEAR']).sum()
yearly_df['PROFIT MARGIN'] = yearly_df['PROFIT'] / yearly_df['REVENUE']
yearly_df

,REVENUE,PROFIT,PROFIT MARGIN
YEAR,,,
2015-01-01,"581,446.00","171,127.34",0.29
2016-01-01,"1,049,612.87","320,765.97",0.31
2017-01-01,"1,451,557.62","525,337.27",0.36
2018-01-01,"2,114,484.91","711,215.80",0.34
2019-01-01,"2,791,129.12","1,096,964.53",0.39
2020-01-01,"3,014,942.44","1,041,249.80",0.35


Create a clustered (grouped) bar chart with revenue and profit by year, excluding year 2021. Hint: Use the color channel to create clusters, just like you would use the color channel to create multiple lines in a line plot. 

In [10]:
melt = yearly_df.reset_index().melt(id_vars=['YEAR'], value_vars=['REVENUE', 'PROFIT'], var_name='INCOME', value_name='AMOUNT')
melt.sort_values(by='YEAR', inplace = True) 
melt

,YEAR,INCOME,AMOUNT
0,2015-01-01,REVENUE,"581,446.00"
6,2015-01-01,PROFIT,"171,127.34"
1,2016-01-01,REVENUE,"1,049,612.87"
7,2016-01-01,PROFIT,"320,765.97"
2,2017-01-01,REVENUE,"1,451,557.62"
8,2017-01-01,PROFIT,"525,337.27"
3,2018-01-01,REVENUE,"2,114,484.91"
9,2018-01-01,PROFIT,"711,215.80"
4,2019-01-01,REVENUE,"2,791,129.12"
10,2019-01-01,PROFIT,"1,096,964.53"


In [11]:
alt.Chart(melt).mark_bar().encode(
    x='INCOME:N',
    y='AMOUNT:Q',
    color='INCOME:N',
    column='YEAR'
)

alt.Chart(...)

Create a stacked bar chart with revenue by year, where the stacks are the two customer types.

In [12]:
df = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Education') | (yearly['CUSTOMER_TYPE'] == 'Tech'), ['YEAR', 'REVENUE','CUSTOMER_TYPE']]
df = df.groupby(['YEAR','CUSTOMER_TYPE']).sum()
df_melt = df.reset_index().melt(id_vars=['YEAR', 'CUSTOMER_TYPE'], value_vars=['REVENUE'], var_name='INCOME', value_name='AMOUNT')
df_melt

,YEAR,CUSTOMER_TYPE,INCOME,AMOUNT
0,2015-01-01,Education,REVENUE,"381,953.62"
1,2015-01-01,Tech,REVENUE,"199,492.38"
2,2016-01-01,Education,REVENUE,"749,545.36"
3,2016-01-01,Tech,REVENUE,"300,067.51"
4,2017-01-01,Education,REVENUE,"905,194.53"
5,2017-01-01,Tech,REVENUE,"546,363.09"
6,2018-01-01,Education,REVENUE,"1,430,265.29"
7,2018-01-01,Tech,REVENUE,"684,219.62"
8,2019-01-01,Education,REVENUE,"1,865,460.20"
9,2019-01-01,Tech,REVENUE,"925,668.92"


In [13]:
alt.Chart(df_melt).mark_bar().encode(
    x='YEAR:T',
    y='AMOUNT:Q',
    color='CUSTOMER_TYPE:N',
    column='INCOME:N'
).properties(
    width=200
)

alt.Chart(...)

Create a line plot with quarterly revenue and profit. Make sure that the x-axis is temporal. 

In [14]:
quarter_df = quarterly.loc[:,['TRANSACTION_PERIOD', 'REVENUE', 'PROFIT']].groupby('TRANSACTION_PERIOD').sum().reset_index()
quarter_df.head()

,TRANSACTION_PERIOD,REVENUE,PROFIT
0,2015-03-31,"117,166.10","35,322.41"
1,2015-06-30,"136,825.95","39,508.29"
2,2015-09-30,"153,914.90","44,812.30"
3,2015-12-31,"173,539.05","51,484.34"
4,2016-03-31,"350,426.84","67,946.76"


In [15]:
revenue = alt.Chart(quarter_df).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('REVENUE:Q'),
    color=alt.value('blue')
).properties(
    title='Quarterly Revenue and Profit'
)

profit = alt.Chart(quarter_df).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('PROFIT:Q'),
    color=alt.value('orange')
)

revenue + profit

alt.LayerChart(...)

## Part Two

Create two tables of yearly revenue change, one for each of the customer types. Include both the difference and the percent change (use diff and pct_change). 

In [16]:
# Education table
edu1 = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Education'), ['YEAR','CUSTOMER_TYPE', 'REVENUE']]
edu1 = edu1.groupby('YEAR').sum()
edu1['DIFFERENCE'] = edu1['REVENUE'].diff().fillna(0)
edu1['REVEVENUE_CHANGE'] = edu1['REVENUE'].pct_change() * 100
edu1['REVEVENUE_CHANGE'] = edu1['REVEVENUE_CHANGE'].fillna(0)
edu1

<ipython-input-16-57f83b56caa6>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  edu1 = edu1.groupby('YEAR').sum()


,REVENUE,DIFFERENCE,REVEVENUE_CHANGE
YEAR,,,
2015-01-01,"381,953.62",0.00,0.00
2016-01-01,"749,545.36","367,591.73",96.24
2017-01-01,"905,194.53","155,649.18",20.77
2018-01-01,"1,430,265.29","525,070.76",58.01
2019-01-01,"1,865,460.20","435,194.91",30.43
2020-01-01,"2,009,986.27","144,526.07",7.75
2021-01-01,"1,848,104.86","-161,881.41",-8.05


In [17]:
# Tech table
tech1 = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Tech'), ['YEAR','CUSTOMER_TYPE', 'REVENUE']]
tech1 = tech1.groupby('YEAR').sum()
tech1['DIFFERENCE'] = tech1['REVENUE'].diff().fillna(0)
tech1['REVEVENUE_CHANGE'] = tech1['REVENUE'].pct_change() * 100
tech1['REVEVENUE_CHANGE'] = tech1['REVEVENUE_CHANGE'].fillna(0)
tech1

<ipython-input-17-36dda85e0db5>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tech1 = tech1.groupby('YEAR').sum()


,REVENUE,DIFFERENCE,REVEVENUE_CHANGE
YEAR,,,
2015-01-01,"199,492.38",0.00,0.00
2016-01-01,"300,067.51","100,575.13",50.42
2017-01-01,"546,363.09","246,295.58",82.08
2018-01-01,"684,219.62","137,856.53",25.23
2019-01-01,"925,668.92","241,449.30",35.29
2020-01-01,"1,004,956.17","79,287.25",8.57
2021-01-01,"919,036.55","-85,919.62",-8.55


For each customer type, calculate the total revenue change from adding new customers for each year. This is the sum of REVENUE for NEW customers for each year. You should end up with a series of revenue values for each customer type.

In [18]:
# Education table
edu = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Education') & (yearly['STATUS'] == 'NEW'), ['YEAR','CUSTOMER_TYPE', 'STATUS', 'REVENUE']]
edu = edu.groupby('YEAR').sum()
edu

<ipython-input-18-f01b068a8698>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  edu = edu.groupby('YEAR').sum()


,REVENUE
YEAR,
2016-01-01,"70,537.68"
2017-01-01,"215,295.47"
2018-01-01,"483,843.85"
2019-01-01,"667,638.31"
2020-01-01,"825,498.35"
2021-01-01,"945,380.24"


In [19]:
# Tech table
tech = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Tech') & (yearly['STATUS'] == 'NEW'), ['YEAR','CUSTOMER_TYPE', 'STATUS', 'REVENUE']]
tech = tech.groupby('YEAR').sum()
tech

<ipython-input-19-5e317b43b969>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tech = tech.groupby('YEAR').sum()


,REVENUE
YEAR,
2016-01-01,"35,539.17"
2017-01-01,"208,310.62"
2018-01-01,"230,114.39"
2019-01-01,"345,573.59"
2020-01-01,"408,896.53"
2021-01-01,"463,975.13"


For each customer type, calculate the total revenue change from existing customers for each year. This is the sum of REVENUE_DIFF for UPSELL and DOWNSELL customers for each year. You should end up with a series of revenue changes for each customer type.

In [20]:
# Education table
edu_diff = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Education') & (yearly['STATUS'] == 'UPSELL') | (yearly['STATUS'] == 'DOWNSELL'), ['YEAR','CUSTOMER_TYPE', 'STATUS', 'REVENUE_DIFF']]
edu_diff = edu_diff.groupby(['YEAR']).sum()
edu_diff

<ipython-input-20-abdeda230d8d>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  edu_diff = edu_diff.groupby(['YEAR']).sum()


,REVENUE_DIFF
YEAR,
2016-01-01,"302,683.77"
2017-01-01,"-70,432.15"
2018-01-01,"47,433.03"
2019-01-01,"37,905.96"
2020-01-01,"-167,675.19"
2021-01-01,"-423,683.78"


In [21]:
# Tech table
tech_diff = yearly.loc[(yearly['CUSTOMER_TYPE'] != 'Education') & ((yearly['STATUS'] == 'UPSELL') | (yearly['STATUS'] == 'DOWNSELL')), ['YEAR', 'CUSTOMER_TYPE', 'STATUS', 'REVENUE_DIFF']]
tech_diff = tech_diff.groupby(['YEAR']).sum()
tech_diff

<ipython-input-21-07ed9ed5422f>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tech_diff = tech_diff.groupby(['YEAR']).sum()


,REVENUE_DIFF
YEAR,
2016-01-01,"69,300.40"
2017-01-01,"50,358.57"
2018-01-01,"-18,755.56"
2019-01-01,"54,490.81"
2020-01-01,"-22,447.43"
2021-01-01,"-139,405.02"


Create two stacked bar charts, one for each customer type, that show the revenue change from adding new customers and the revenue change from existing customers.

For each chart, you will need to combine the series from the last two questions. Create a dataframe with the two series and the years. Then either melt or use an Altair fold transform to get the data in long form and then create your chart.


In [22]:
# Education
edu_stacked = pd.merge(edu, edu_diff, on='YEAR', how='outer').reset_index()
edu_stacked_melt = pd.melt(edu_stacked, id_vars=['YEAR'], var_name='REVENUE_TYPE', value_name='AMOUNT')

alt.Chart(edu_stacked_melt).mark_bar().encode(
    x='YEAR:T',
    y='AMOUNT:Q',
    color='REVENUE_TYPE:N'
).properties(
    width=200
)

alt.Chart(...)

In [23]:
# Tech
tech_stacked = pd.merge(tech, tech_diff, on='YEAR', how='outer').reset_index()
tech_stacked_melt = pd.melt(tech_stacked, id_vars=['YEAR'], var_name='REVENUE_TYPE', value_name='AMOUNT')

alt.Chart(tech_stacked_melt).mark_bar().encode(
    x='YEAR:T',
    y='AMOUNT:Q',
    color='REVENUE_TYPE:N'
).properties(
    width=200
)

alt.Chart(...)

Use the information from the past questions to answer this question. 

For each customer type, what percentage of growth (revenue change) has come from the acquisition of new customers and what
percentage has come from the upsell/downsell of existing customers? 

You should end up with a dataframe for each customer type. Each dataframe should have two columns: the percentage from upsell/downsell and the percentage from new customers. The rows should be the years.

In [24]:
# Education growth
edu_growth = pd.merge(edu1, edu_stacked, on='YEAR', how='outer')
edu_growth = edu_growth.drop(columns=['REVENUE_x', 'REVEVENUE_CHANGE'])
edu_growth = edu_growth.rename(columns={'REVENUE_y': 'NEW', 'REVENUE_DIFF': 'UPSELL/DOWNSELL'})
edu_growth['NEW_PCT'] = edu_growth['NEW'] / edu_growth['DIFFERENCE'] * 100
edu_growth['UPSELL/DOWNSELL_PCT'] = edu_growth['UPSELL/DOWNSELL'] / edu_growth['DIFFERENCE'] * 100
edu_growth

,YEAR,DIFFERENCE,NEW,UPSELL/DOWNSELL,NEW_PCT,UPSELL/DOWNSELL_PCT
0,2015-01-01,0.00,NaN,NaN,NaN,NaN
1,2016-01-01,"367,591.73","70,537.68","302,683.77",19.19,82.34
2,2017-01-01,"155,649.18","215,295.47","-70,432.15",138.32,-45.25
3,2018-01-01,"525,070.76","483,843.85","47,433.03",92.15,9.03
4,2019-01-01,"435,194.91","667,638.31","37,905.96",153.41,8.71
5,2020-01-01,"144,526.07","825,498.35","-167,675.19",571.18,-116.02
6,2021-01-01,"-161,881.41","945,380.24","-423,683.78",-584.00,261.72


In [25]:
# Tech growth
tech_growth = pd.merge(tech1, tech_stacked, on='YEAR', how='outer')
tech_growth = tech_growth.drop(columns=['REVENUE_x', 'REVEVENUE_CHANGE'])
tech_growth = tech_growth.rename(columns={'REVENUE_y': 'NEW', 'REVENUE_DIFF': 'UPSELL/DOWNSELL'})
tech_growth['NEW_PCT'] = tech_growth['NEW'] / tech_growth['DIFFERENCE'] * 100
tech_growth['UPSELL/DOWNSELL_PCT'] = tech_growth['UPSELL/DOWNSELL'] / tech_growth['DIFFERENCE'] * 100
tech_growth.drop(index=0)

,YEAR,DIFFERENCE,NEW,UPSELL/DOWNSELL,NEW_PCT,UPSELL/DOWNSELL_PCT
1,2016-01-01,"100,575.13","35,539.17","69,300.40",35.34,68.90
2,2017-01-01,"246,295.58","208,310.62","50,358.57",84.58,20.45
3,2018-01-01,"137,856.53","230,114.39","-18,755.56",166.92,-13.61
4,2019-01-01,"241,449.30","345,573.59","54,490.81",143.12,22.57
5,2020-01-01,"79,287.25","408,896.53","-22,447.43",515.72,-28.31
6,2021-01-01,"-85,919.62","463,975.13","-139,405.02",-540.01,162.25


For each customer type, calculate the yearly revenue lost from customers churning. This is the sum of REVENUE of customers for which CHURN is true. Put these two series into a dataframe.

In [26]:
edu_loss = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Education') & (yearly['CHURN'] == True), ['YEAR','CUSTOMER_TYPE', 'CHURN', 'REVENUE']]
edu_loss = edu_loss.groupby('YEAR').sum()
edu_loss

<ipython-input-26-5c2c15549a3c>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  edu_loss = edu_loss.groupby('YEAR').sum()


,CHURN,REVENUE
YEAR,,
2015-01-01,30,"8,249.63"
2016-01-01,37,"11,789.55"
2017-01-01,472,"146,803.30"
2018-01-01,1282,"351,627.70"
2019-01-01,2780,"647,135.06"
2020-01-01,4610,"887,922.63"


In [27]:
tech_loss = yearly.loc[(yearly['CUSTOMER_TYPE'] == 'Tech') & (yearly['CHURN'] == True), ['YEAR','CUSTOMER_TYPE', 'CHURN', 'REVENUE']]
tech_loss = tech_loss.groupby('YEAR').sum()
tech_loss

<ipython-input-27-642715ca1ca3>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tech_loss = tech_loss.groupby('YEAR').sum()


,CHURN,REVENUE
YEAR,,
2015-01-01,16,"4,264.44"
2016-01-01,31,"12,373.61"
2017-01-01,234,"73,502.30"
2018-01-01,615,"158,615.10"
2019-01-01,1326,"307,161.85"
2020-01-01,2189,"410,489.73"


Create a single stacked bar chart showing the revenue lost for both customer types over the years.

In [28]:
combined = pd.merge(edu_loss, tech_loss, on='YEAR', how='outer')
combined = combined.rename(columns={'REVENUE_x': 'EDU_LOSS', 'REVENUE_y': 'TECH_LOSS'}).drop(columns = ['CHURN_x','CHURN_y'])
combined

,EDU_LOSS,TECH_LOSS
YEAR,,
2015-01-01,"8,249.63","4,264.44"
2016-01-01,"11,789.55","12,373.61"
2017-01-01,"146,803.30","73,502.30"
2018-01-01,"351,627.70","158,615.10"
2019-01-01,"647,135.06","307,161.85"
2020-01-01,"887,922.63","410,489.73"


In [29]:
combined_melt = combined.reset_index().melt(id_vars=['YEAR'], value_vars=['EDU_LOSS', 'TECH_LOSS'], var_name='LOSS_TYPE', value_name='LOSS_AMOUNT')

alt.Chart(combined_melt).mark_bar().encode(
    x='YEAR:T',
    y='LOSS_AMOUNT:Q',
    color='LOSS_TYPE:N'
).properties(
    width=200
)

alt.Chart(...)

## Part Three

Create a line plot of quarterly revenue change from new customers and existing customers (this line plot should have 2 lines, new and existing).

In [30]:
# From new customers
new = quarterly.loc[(quarterly['STATUS'] == 'NEW'), ['TRANSACTION_PERIOD', 'STATUS', 'REVENUE']]
new = new.groupby('TRANSACTION_PERIOD').sum()

# From existing customers
existing = quarterly.loc[(quarterly['STATUS'] == 'UPSELL') | (quarterly['STATUS'] == 'DOWNSELL'), ['TRANSACTION_PERIOD', 'STATUS', 'REVENUE_DIFF']]
existing = existing.groupby(['TRANSACTION_PERIOD']).sum()

quarter_change = pd.merge(new, existing, on='TRANSACTION_PERIOD', how='outer')
#quarter_change = quarter_change.drop(index = '2015-03-31')
line_plot = quarter_change.reset_index()

revenue = alt.Chart(line_plot).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('REVENUE:Q'),
    color=alt.value('green')
).properties(
    title='Quarterly Revenue Change from New vs Existing Customers'
)

difference = alt.Chart(line_plot).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('REVENUE_DIFF:Q'),
    color=alt.value('orange')
)

revenue + difference

<ipython-input-30-944e00418a80>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new = new.groupby('TRANSACTION_PERIOD').sum()
<ipython-input-30-944e00418a80>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  existing = existing.groupby(['TRANSACTION_PERIOD']).sum()


alt.LayerChart(...)

Create a line plot of quarterly revenue lost from customers churning.

In [32]:
churn_loss = quarterly.loc[(quarterly['CHURN'] == True), ['TRANSACTION_PERIOD','CUSTOMER_TYPE', 'CHURN', 'REVENUE']]
churn_loss = churn_loss.groupby('TRANSACTION_PERIOD').sum()
line_plot2 = churn_loss.reset_index()

alt.Chart(line_plot2).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('REVENUE:Q'),
    color=alt.value('red')
).properties(
    title='Quarterly Revenue Loss from Customers Churning'
)

<ipython-input-32-7867764b9257>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  churn_loss = churn_loss.groupby('TRANSACTION_PERIOD').sum()


alt.Chart(...)

Create a line plot of number of customers lost quarterly. Hint use groupby and count.

In [35]:
churn_loss2 = quarterly.loc[(quarterly['CHURN'] == True), ['TRANSACTION_PERIOD','CUSTOMER_TYPE', 'CHURN', 'REVENUE']]
churn_loss2 = churn_loss2.groupby('TRANSACTION_PERIOD').count()
line_plot3 = churn_loss2.reset_index()

alt.Chart(line_plot3).mark_line().encode(
    x=alt.X('TRANSACTION_PERIOD:T'),
    y=alt.Y('REVENUE:Q'),
    color=alt.value('red')
).properties(
    title='Quarterly Customer Loss from Churning'
)

alt.Chart(...)

## Part Four

The IS_TREATMENT_FLAG column indicates whether a customer received a new promotion. The promotion is designed to increase revenue from the customer.

The promotion occurred in year 2021. So, only in year 2021 do customers have True or False in that column, otherwise it is nan. The value is True for those who received the promotion and False for those who did not.

Using the yearly data, determine by what percentage revenue changed from 2020 to 2021 across the two groups of customers (those who received the promotion and those who did not). 

Hint: Pull out the two groups (True and False) from 2021. For each group, use isin to find the corresponding customers in 2020, and then use isin again to pull out just those customers from 2021. Now you have the 2020 and 2021 values for those customers.

In [38]:
promotion = yearly.loc[yearly['IS_TREATMENT_FLAG'].isin([True, False]), ['CUSTOMER_ID','YEAR','REVENUE', 'CUSTOMER_TYPE', 'IS_TREATMENT_FLAG','STATUS','REVENUE_DIFF','CHURN']]
customers2020 = yearly.loc[yearly['IS_TREATMENT_FLAG'].isin([True, False]), ['CUSTOMER_ID','YEAR','REVENUE', 'CUSTOMER_TYPE', 'IS_TREATMENT_FLAG','STATUS','REVENUE_DIFF','CHURN']]

,CUSTOMER_ID,YEAR,REVENUE,CUSTOMER_TYPE,IS_TREATMENT_FLAG,STATUS,REVENUE_DIFF,CHURN
7,1,2021-01-01,"1,539.93",Education,False,UPSELL,259.36,False
29,5,2021-01-01,"1,178.05",Education,False,DOWNSELL,-101.10,False
69,16,2021-01-01,400.32,Education,False,DOWNSELL,-454.65,False
76,17,2021-01-01,270.71,Tech,False,DOWNSELL,"-1,010.19",False
83,18,2021-01-01,459.23,Education,True,DOWNSELL,-444.48,False
...,...,...,...,...,...,...,...,...
51047,29299,2021-01-01,117.63,Education,True,NEW,0.00,False
51048,29300,2021-01-01,79.00,Tech,True,NEW,0.00,False
51049,29301,2021-01-01,80.36,Tech,False,NEW,0.00,False
51050,29302,2021-01-01,78.73,Tech,False,NEW,0.00,False


Do the same analysis but break the data into customer types first. This way we can see if the promotion had different effects for different customer types.